In [2]:
# #pip  install alibi-detect --user
# # # !pip  install tensorflow
# !pip install catboost
# !pip install xgboost
!pip install category_encoders
# # # !pip install torch torchvision torchaudio
# !pip install lightgbm
# !pip install verta
!pip install minio

!pip install git+https://github.com/amirgholipour/alibi-detect/tree/bugs_free_alibi

!pip install ipynbname
!pip install python-opencv
!pip install mlflow

     |████████████████████████████████| 80 kB 27.5 MB/s eta 0:00:01
     |████████████████████████████████| 15.8 MB 96.9 MB/s eta 0:00:01
     |████████████████████████████████| 28.4 MB 91.0 MB/s eta 0:00:01
     |████████████████████████████████| 25.8 MB 88.1 MB/s eta 0:00:01
     |████████████████████████████████| 9.4 MB 76.9 MB/s eta 0:00:01
     |████████████████████████████████| 233 kB 65.7 MB/s eta 0:00:01
     |████████████████████████████████| 11.5 MB 122.3 MB/s eta 0:00:01
     |████████████████████████████████| 303 kB 110.1 MB/s eta 0:00:01
  Cloning https://github.com/amirgholipour/alibi-detect.git to /tmp/pip-req-build-rnyxlvka
  Running command git clone -q https://github.com/amirgholipour/alibi-detect.git /tmp/pip-req-build-rnyxlvka
  Resolved https://github.com/amirgholipour/alibi-detect.git to commit ae248be3068dc17a46f3ad4359f8b0a12a908007
     |████████████████████████████████| 10.3 MB 24.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 111.2 MB/s eta

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |█████████████████████████████▎  | 420.1 MB 160.3 MB/s eta 0:00:01

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 4.8 MB 111.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 86.1 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 110.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 105.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 105.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 101.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 108.0 MB/s eta 0:00:01
     |████████████████████████████████| 171 kB 85.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.9 MB 106.6 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 106.0 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 137.2 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 97.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 119.8 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 107.6 MB/s eta 0:00:01
     |████████████████████

In [4]:
!pip install git+https://github.com/amirgholipour/alibi-detect/tree/bugs_free_alibi


  Cloning https://github.com/amirgholipour/alibi-detect/tree/bugs_free_alibi to /tmp/pip-req-build-ooexe456
  Running command git clone -q https://github.com/amirgholipour/alibi-detect/tree/bugs_free_alibi /tmp/pip-req-build-ooexe456
  fatal: repository 'https://github.com/amirgholipour/alibi-detect/tree/bugs_free_alibi/' not found
ERROR: Command errored out with exit status 128: git clone -q https://github.com/amirgholipour/alibi-detect/tree/bugs_free_alibi /tmp/pip-req-build-ooexe456 Check the logs for full command output.


# Load libraries

In [3]:


# import alibi

from alibi_detect.cd import TabularDrift,MMDDrift
# from alibi_detect.utils.saving import save_detector, load_detector
import os

import pandas as pd
import numpy as np
import matplotlib as mat
import matplotlib.pyplot as plt    
# import seaborn as sns
import category_encoders as ce
import joblib
from datetime import datetime

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from minio import Minio
import mlflow
import subprocess
import ipynbname
# from verta import Client
# from minio.error import ResponseError

# import pickle
import warnings

ImportError: libGL.so.1: cannot open shared object file: No such file or directory

# MLflow configuration
Configure mlflow

In [ ]:
# # from verta.utils import ModelAPI

# # Connect to local MLflow tracking server
# mlflow.set_tracking_uri(HOST)

# # Set the experiment name...
# mlflow.set_experiment(EXPERIMENT_NAME)

# mlflow.sklearn.autolog(log_input_examples=True)
# mlflow.autolog(log_input_examples=True)

In [6]:
def fetch_logged_data(run_id):
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

In [7]:
## TODO move it to a library
import subprocess
import ipynbname

def get_git_revision_hash():
    return subprocess.check_output(['git', 'rev-parse', 'HEAD'])

def get_git_revision_short_hash():
    return subprocess.check_output(['git', 'rev-parse', '--short', 'HEAD'])

def get_git_remote():
    return subprocess.check_output(['git', 'config', '--get', 'remote.origin.url'])

def get_git_user():
    return subprocess.check_output(['git', 'config', 'user.name'])

def get_git_branch():
    return subprocess.check_output(['git', 'branch', '--show-current'])

def get_pip_freeze():
    return subprocess.check_output(['pip', 'freeze']).splitlines()


def record_details(mlflow):
    """
    This method is the anchor poijt and more activiteis will go in it
    :param mlflow:
    :return:
    """
    with open("pip_freeze.txt", "wb") as file:
        for line in get_pip_freeze():
            file.write(line)
            file.write(bytes("\n", "UTF-8"))
    mlflow.log_artifact("pip_freeze.txt")
    file.close()
    os.remove("pip_freeze.txt")


def mlflow_grid_search(methodtoexecute, methodarguments):
    with mlflow.start_run(tags= {
        "mlflow.source.git.commit" : get_git_revision_hash() ,
        "mlflow.user": get_git_user(),
        "mlflow.source.git.repoURL": get_git_remote(),
        "git_remote": get_git_remote(),
        "mlflow.source.git.branch": get_git_branch(),
        "mlflow.docker.image.name": os.getenv("JUPYTER_IMAGE", "LOCAL"),
        "mlflow.source.type": "NOTEBOOK",
        "mlflow.source.name": ipynbname.name()
    }) as run:
        methodtoexecute(**methodarguments)
        record_details(mlflow)

    return run

In [8]:


def get_s3_server():
    minioClient = Minio('minio-ml-workshop:9000',
                    access_key='minio',
                    secret_key='minio123',
                    secure=False)


# Read data

In [9]:
data = pd.read_csv('../../data/raw/data.csv')
data.head(5)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,148,Male,0,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,45.65,45.65,Yes
1,463,Male,0,Yes,Yes,4,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,101.15,385.90,Yes
2,471,Female,1,No,No,17,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,20.65,330.60,No
3,496,Male,0,No,No,22,No,No phone service,DSL,No,...,Yes,No,No,Yes,One year,Yes,Bank transfer (automatic),43.75,903.60,Yes
4,833,Female,0,Yes,Yes,70,Yes,No,DSL,Yes,...,Yes,Yes,No,Yes,One year,No,Credit card (automatic),74.10,5222.30,No


# Data cleaning

Map the churn data to numeric number.

Replace missing value.

Remove churn and Customer ID.

In [10]:
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

data.replace(" ", np.nan, inplace=True)

data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

mean = data['TotalCharges'].mean()
data.fillna(mean, inplace=True)
final_set = data.drop(['Churn', 'customerID'], axis=1)
final_set.head(5)
labels = data['Churn']


final_set.head(5)





,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Male,0,No,No,1,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,45.65,45.65
1,Male,0,Yes,Yes,4,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,101.15,385.90
2,Female,1,No,No,17,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,20.65,330.60
3,Male,0,No,No,22,No,No phone service,DSL,No,Yes,Yes,No,No,Yes,One year,Yes,Bank transfer (automatic),43.75,903.60
4,Female,0,Yes,Yes,70,Yes,No,DSL,Yes,Yes,Yes,Yes,No,Yes,One year,No,Credit card (automatic),74.10,5222.30


In [11]:


labels = data['Churn']
X_train, X_test, Y_train, Y_test = train_test_split(final_set, labels, test_size=0.3)
print ('Training Data Shape',X_train.shape, Y_train.shape)
print ('Testing Data Shape',X_test.shape, Y_test.shape)

Y = data['Churn']
X = final_set
joblib.dump(final_set.columns, 'CustomerChurnDriftFeaturesColumns.pkl')



Training Data Shape (4930, 19) (4930,)
Testing Data Shape (2113, 19) (2113,)


['CustomerChurnDriftFeaturesColumns.pkl']


# Define a Drift Model.

### Find the categorical columns.

In [12]:
numCols = X_train.select_dtypes("number").columns
catCols = X_train.select_dtypes("object").columns
numCols= list(set(numCols))
catCols= list(set(catCols))
print (numCols,catCols)

['MonthlyCharges', 'TotalCharges', 'tenure', 'SeniorCitizen'] ['Partner', 'Dependents', 'InternetService', 'StreamingTV', 'DeviceProtection', 'PaperlessBilling', 'OnlineBackup', 'StreamingMovies', 'MultipleLines', 'PhoneService', 'gender', 'PaymentMethod', 'Contract', 'TechSupport', 'OnlineSecurity']


### Create a dict for categorical feature

In [13]:

categories_per_feature = {}
for i in range (len(final_set.columns)):
    if final_set.columns[i] in catCols:
        categories_per_feature[i] = None
categories_per_feature

{0: None,
 2: None,
 3: None,
 5: None,
 6: None,
 7: None,
 8: None,
 9: None,
 10: None,
 11: None,
 12: None,
 13: None,
 14: None,
 15: None,
 16: None}

In [16]:
cd = TabularDrift(X_train.values, p_val=.05, categories_per_feature=categories_per_feature)


In [17]:

joblib.dump(cd, 'model1.pkl')


PicklingError: Can't pickle <class 'alibi_detect.cd.tabular.TabularDrift'>: it's not found as alibi_detect.cd.tabular.TabularDrift

In [ ]:
# save_detector(cd, 'CustomerChurnDriftPredictor')
# cd1 = load_detector('CustomerChurnDriftPredictor')
latest_run_id = mlflow.search_runs().sort_values('start_time').run_id.values[-1]
with mlflow.start_run(run_id=latest_run_id):
  mlflow.log_artifacts('model', 'model/model')

In [84]:
with mlflow.start_run(tags= {
        "mlflow.source.git.commit" : get_git_revision_hash() ,
        "mlflow.user": get_git_user(),
        "mlflow.source.git.repoURL": get_git_remote(),
        "git_remote": get_git_remote(),
        "mlflow.source.git.branch": get_git_branch(),
        "mlflow.docker.image.name": os.getenv("JUPYTER_IMAGE", "LOCAL"),
        "mlflow.source.type": "NOTEBOOK",
#         "mlflow.source.name": ipynbname.name()
    }) as run:
        cd = TabularDrift(X_train.values, p_val=.05, categories_per_feature=categories_per_feature)
#         mlflow.pyfunc.add_to_model(cd, cd.predict, data=categories_per_feature, code=None, env=None)
        mlflow.pyfunc.log_model('/tmp',  python_model=cd)

        record_details(mlflow)

MlflowException: `python_model` must be a subclass of `PythonModel`. Instead, found an object of type: <class 'alibi_detect.cd.tabular.TabularDrift'>

In [74]:
params, metrics, tags, artifacts = fetch_logged_data(run.info.run_id)

In [76]:
metrics

{}

In [59]:
import mlflow

mlflow.start_run()
run = mlflow.active_run()
print("Active run_id: {}".format(run.info.run_id))
mlflow.log_param("my", "param")
mlflow.log_metric("score", 100)
mlflow.end_run()

Active run_id: b02181f779544bdea30caa714aaaf7a4


In [52]:
with mlflow.start_run() as run:
    mlflow.log_param("my", "param")
    mlflow.log_metric("score", 100)
    
    

In [50]:

with mlflow.start_run() as run:
    cd = TabularDrift(X_train.values, p_val=.05, categories_per_feature=categories_per_feature)

In [17]:
data_for_check = X_test.copy()
list_dist = []
list_dist_d= []
list_dist_nd= []

list_drift = []
list_drift_index = []
idx =0
cc = 0
for index, row in data_for_check.iterrows():
    cd_preds = cd.predict(data_for_check.iloc[idx].values.reshape(1,data_for_check.shape[1]))
    if cd_preds['data']['is_drift']==1:
        list_drift_index.append(idx)
        list_dist_d.append(np.min(cd_preds['data']['p_val']))

    else:
        list_dist_nd.append(np.min(cd_preds['data']['p_val']))

    list_drift.append(cd_preds['data']['is_drift'])
    list_dist.append(np.mean(cd_preds['data']['p_val']))
    idx +=1
len (list_drift_index)

9

9

In [11]:
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

data.replace(" ", np.nan, inplace=True)

data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

mean = data['TotalCharges'].mean()
data.fillna(mean, inplace=True)


names = ['gender', 'Partner', 'Dependents', 'PhoneService', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
# for column in names:
#     labelencoder(column)
data_enc = data
data_enc = data_enc.drop(['Churn', 'customerID'], axis=1)
enc = ce.ordinal.OrdinalEncoder(cols=names)
enc.fit(data_enc)
# ###
# filename_oe = 'OrdinalEncoder_drift.sav'
# pickle.dump(enc, open(filename_oe, 'wb'))

# # some time later...

# # load the model from disk
# enc = pickle.load(open(filename_oe, 'rb'))
# ###
labelled_set = enc.transform(data_enc)

names = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport']

ohe = ce.OneHotEncoder(cols=names)
data_ohe = data
data_ohe = data_ohe.drop(['Churn', 'customerID'], axis=1)
ohe.fit(data_ohe)
# ###
# filename_ohe = 'OneHotEncoder_drift.sav'
# pickle.dump(ohe, open(filename_ohe, 'wb'))

# # some time later...

# # load the model from disk
# ohe = pickle.load(open(filename_ohe, 'rb'))
###
final_set = ohe.transform(labelled_set)
columns_name =final_set.columns
final_set.head(5)

/opt/app-root/lib64/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines_1,MultipleLines_2,MultipleLines_3,InternetService_1,...,Contract_1,Contract_2,Contract_3,PaperlessBilling,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3,PaymentMethod_4,MonthlyCharges,TotalCharges
0,1,0,1,1,1,1,1,0,0,1,...,1,0,0,1,1,0,0,0,45.65,45.65
1,1,0,2,2,4,1,0,1,0,0,...,1,0,0,2,1,0,0,0,101.15,385.90
2,2,1,1,1,17,1,1,0,0,0,...,0,1,0,2,0,1,0,0,20.65,330.60
3,1,0,1,1,22,2,0,0,1,1,...,0,1,0,1,0,0,1,0,43.75,903.60
4,2,0,2,2,70,1,1,0,0,1,...,0,1,0,2,0,0,0,1,74.10,5222.30


In [12]:
from sklearn.model_selection import train_test_split
labels = data['Churn']
X_train, X_test, Y_train, Y_test = train_test_split(final_set, labels, test_size=0.3)

Y = data['Churn']
X = final_set

In [13]:
from tensorflow.keras.layers import Dense, InputLayer, Input,Conv1D,Flatten,Embedding
import tensorflow as tf


n_features = X_train.shape[1]

proj = tf.keras.Sequential(
  [
      InputLayer(input_shape=( n_features,1), name="encode1"),

      Conv1D(32, 4, strides=1, padding='same', activation=tf.nn.relu),
      Conv1D(64, 4, strides=1, padding='same', activation=tf.nn.relu),
      Conv1D(4, 4, strides=1, padding='same', activation=tf.nn.relu),
      Flatten(),
  ]
)

proj.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 36, 32)            160       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 36, 64)            8256      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 36, 4)             1028      
_________________________________________________________________
flatten (Flatten)            (None, 144)               0         
Total params: 9,444
Trainable params: 9,444
Non-trainable params: 0
_________________________________________________________________


In [14]:
from alibi_detect.utils.tensorflow.kernels import DeepKernel
kernel = DeepKernel(proj, eps=0.01)


In [15]:
# x = X_train.values.reshape(X_train.values.shape[0], X_train.values.shape[1], 1)
# x_t = X_test.values.reshape(X_test.values.shape[0], X_test.values.shape[1], 1).astype('float32')
x = X_train.values.reshape(X_train.values.shape[0],n_features,1).astype('float32')
x_t = X_test.values.reshape(X_test.values.shape[0],n_features,1).astype('float32')
x_t.shape



(2113, 36, 1)

In [16]:

from alibi_detect.cd import LearnedKernelDrift
cd = LearnedKernelDrift(x, kernel, backend='tensorflow', p_val=.05, epochs=1)

In [17]:
x_t0 = x_t[0].reshape(1,n_features,1).astype('float32')

x_t0.shape


(1, 36, 1)

In [18]:
data_for_check = X_test.copy()

cd_preds = cd.predict(x_t0)

ZeroDivisionError: division by zero

In [ ]:
data_for_check = X_test.copy()
list_dist = []
list_dist_d= []
list_dist_nd= []

list_drift = []
list_drift_index = []
idx =0
cc = 0
for index, row in data_for_check.iterrows(): 
    x_t0 = x_t[idx].reshape(1,n_features,1).astype('float32')
    cd_preds = cd.predict(x_t0)
    # cd_preds = cd.predict(data_for_check.iloc[idx].values.reshape(1,data_for_check.shape[1]))
    if cd_preds['data']['is_drift']==1:
        list_drift_index.append(idx)
        list_dist_d.append(np.min(cd_preds['data']['p_val']))
        
    else:
        list_dist_nd.append(np.min(cd_preds['data']['p_val']))

    list_drift.append(cd_preds['data']['is_drift'])
    list_dist.append(np.mean(cd_preds['data']['p_val']))
    idx +=1
len (list_drift_index)